<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Terraclimate downscaling

_by Oriana Chegwidden (CarbonPlan), December 15, 2020_

This notebook evaluates the downscaling of CMIP6 variables against the obs
Terraclimate dataset.


In [ ]:
%load_ext autoreload
%autoreload 2

from cmip6_downscaling.analysis import load, metrics, plot
import dask
import os
import matplotlib.pyplot as plt

In [ ]:
from dask_gateway import Gateway

gateway = Gateway()
options = gateway.cluster_options()
options.worker_cores = 2
options.worker_memory = 24
cluster = gateway.new_cluster(cluster_options=options)
cluster.adapt(minimum=1, maximum=10)
cluster

In [ ]:
client = cluster.get_client()
client

In [ ]:
# load obs data
obs = load.load_obs()
# load cmip data
gcm_bias_corrected = True
gcm_name = "CanESM5"
gcm_scenario = "historical"
gcm_ensemble_member = "r10i1p1f1"

gcm = load.load_cmip(
    model=gcm_name,
    scenario=gcm_scenario,
    member=gcm_ensemble_member,
    bias_corrected=gcm_bias_corrected,
)
# we only really have a few variables that we can compare across
# though we can compare vapor pressure by doing a combo of rh and pressure
if gcm_bias_corrected:
    variables_to_plot = [
        "ppt",
        "rh",
        "tmax",
        "tmin",
        "srad",
        "pdsi",
        "vap",
        "pet",
    ]

else:
    variables_to_plot = ["ppt", "rh", "tmax", "tmin", "srad"]

# calculate a metric from existing ones
# make a plot comparing cmip and obs

Define some periods over which we'll conduct our analyses.


In [ ]:
time_slice_dict = {
    "before calibration": slice("1955", "1969"),
    "calibration": slice("1970", "1999"),
    "after calibration": slice("2000", "2014"),
}

In [ ]:
period_of_interest = "calibration"

In [ ]:
results_obs = metrics.calc(
    obs[variables_to_plot].sel(time=time_slice_dict[period_of_interest]),
    compute=True,
)

In [ ]:
results_gcm = metrics.calc(
    gcm[variables_to_plot].sel(time=time_slice_dict[period_of_interest]),
    compute=True,
)

The goal of these analyses is to evaluate the effect of different steps in the
modeling chain. By teasing out where bias is introduced or where the
distribution gets distorted, we'll better understand our output met datasets.


Future climate plotting - within each of the historical timeseries, what
influence does a future climate have on each of the variables. Spatial plots


Make a multivariable map plot with columns being (left) obs (center) gcm and
(right) the difference. Rows are the different variables selected via
`variables_to_plot`. We'll do this a few times to see how the bias-correction
affects things.

Things we'll be sure to include in our plotting functionality.

- averaging time period
- gcm/ensemble/historical
- standardize colorbars for the obs/historical


Some of the variables throw wierd numbers so for plotting we'll constrain them
to a reasonable limit.


In [ ]:
limits_dict = {"pdsi": {"abs": (-2, 2), "diff": (-0.5, 0.5)}}

For our plotting we'll want to include a human-readable representation of
whether or not the dataset was bias-corrected.


In [ ]:
bias_correction_name_dict = {True: "bias-corrected", False: "raw"}

First we'll check how different the bias-correction produces GCM variables that
match Terraclimate. We'll check it across the calibration period first.


In [ ]:
fig, axarr = plot.plot_time_mean(
    results_obs["time_mean"],
    results_gcm["time_mean"],
    diff=True,
    limits_dict=limits_dict,
    figsize=(15, 20),
    ds1_name="TerraClimate",
    ds2_name="{}-{}-{}".format(
        gcm_name,
        gcm_ensemble_member,
        bias_correction_name_dict[gcm_bias_corrected],
    ),
    title='Comparing met datasets for {} period ({}-{})\naka "how far are we off even after bias-correction"'.format(
        period_of_interest,
        time_slice_dict[period_of_interest].start,
        time_slice_dict[period_of_interest].stop,
    ),
    savefig="/home/jovyan/figures/timemean-{}-"
    "TerraClimate-{}-{}.png".format(
        period_of_interest,
        gcm_name,
        bias_correction_name_dict[gcm_bias_corrected],
    ),
)

Now let's check it for the other climate periods. To do this we'll have to
reload the datasets with a different slice selected and recalculate the metrics.


Then let's repeat the whole thing (all climate periods) but this time doing it
thinking of ds1 as gcm-raw and ds2 being gcm-biascorrected


Now let's also do some seasonal mean plotting. Maybe the hub will behave better
for this one.


In [ ]:
fig, axarr = plot_seasonal_mean(
    results_obs["seasonal_mean"].groupby("time.month").mean(),
    results_gcm["seasonal_mean"].groupby("time.month").mean(),
    limits_dict=limits_dict,
)

Spatial autocorrelation analyses Things to think about:

- what weighting scheme
- how many spatiotemporal lags to have


In [ ]:
import pysal
from esda.moran import Moran

mi = Moran(y, w)